### Importing libs and connecting to the database

In [ ]:
import pymongo
import json
import math
import datetime
from pymongo import MongoClient, InsertOne
from pprint import pprint # for pretty print
from pymongo import ASCENDING, DESCENDING
from pymongo import GEO2D
from pymongo import GEOSPHERE 

CONNECTION_STRING = "mongodb://localhost:27017/"
DATABASE = "citytransportweb"

client = pymongo.MongoClient(CONNECTION_STRING)
db = client[DATABASE]
client.drop_database(DATABASE) # clear database when its needed

### Importing data from mysql database

In [ ]:
# Данные были зарание экспортированы из mysql в json файлы
list_of_data_files = [r"a_stop.json", r"parking.json", r"parking_type.json", r"public_vehicle.json",\
                      r"rented_vehicle.json", r"route.json", r"route_stop.json", r"sensors_data.json",\
                      r"status.json", r"stop_type.json", r"transport.json", r"transport_type.json",\
                      r"trip.json", r"trip_type.json", r"validator_s_data.json", r"vehicle_rent_status_data.json"]
list_of_data_names = ["a_stop", "parking", "parking_type", "public_vehicle", "rented_vehicle", "route", "route_stop",\
                      "sensors_data", "status", "stop_type", "transport", "transport_type", "trip", "trip_type", "validator_s_data",\
                      "vehicle_rent_status_data"]

In [ ]:
for filepath, filename in zip(list_of_data_files, list_of_data_names):

  сollection = db[filename]
  requesting = []

  with open(filepath, 'r', encoding='utf-8') as file:
    file_data = json.load(file)
  
  if isinstance(file_data, list):
    сollection.insert_many(file_data)  
  else:
    сollection.insert_one(file_data)

In [ ]:
db.list_collection_names()

['public_vehicle',
 'rented_vehicle',
 'transport_type',
 'route',
 'transport',
 'trip_type',
 'route_stop',
 'sensors_data',
 'trip',
 'parking_type',
 'validator_s_data',
 'parking',
 'a_stop',
 'stop_type',
 'status',
 'vehicle_rent_status_data']

In [ ]:
# Данные vehicle_rent_status_data сохранились с ошибкой (поле Status_id принимает значения None). Востановим эти данные с помощью поля Card_id
print("Уникальные значения:", db.vehicle_rent_status_data.distinct("Status_id"))
# Исправление
db.vehicle_rent_status_data.update_many({"Card_id": {"$ne": None}}, {"$set": { "Status_id" : True}})
db.vehicle_rent_status_data.update_many({"Card_id": {"$eq": None}}, {"$set": { "Status_id" : False}})
print("Уникальные значения после исправления:", db.vehicle_rent_status_data.distinct("Status_id"))

Уникальные значения: [None]
Уникальные значения после исправления: [False, True]


### Data statistics

In [ ]:
# print database statistics
pprint(db.command("dbstats"))

{'avgObjSize': 147.26402427034466,
 'collections': 16,
 'dataSize': 5242452.0,
 'db': 'citytransportweb',
 'freeStorageSize': 0.0,
 'fsTotalSize': 510118756352.0,
 'fsUsedSize': 434581102592.0,
 'indexFreeStorageSize': 0.0,
 'indexSize': 65536.0,
 'indexes': 16,
 'objects': 35599,
 'ok': 1.0,
 'scaleFactor': 1.0,
 'storageSize': 65536.0,
 'totalFreeStorageSize': 0.0,
 'totalSize': 131072.0,
 'views': 0}


In [ ]:
# for collection in list_of_data_names:
#   # print collection statistics
#   pprint(db.command("collstats", collection))
#   break

### Queries

In [ ]:
### Query 1
### Найти все транспортные средства которые брали в аренду 25.05.2020
result = db.vehicle_rent_status_data.aggregate(
    [{"$match": {"$and": [{"Departure_waiting_time": {"$gte": '2020-05-25 00:00:00', "$lt": '2020-05-26 00:00:00'}}, {"Status_id": True}]}},
    ]
)

for doc in result:
  pprint(doc)

{'Average_speed': 85,
 'Card_id': 8450221817428453,
 'Departure_waiting_time': '2020-05-25 01:26:08',
 'Rented_vehicle_id': 7,
 'Status_id': True,
 'Vehicle_id': 11,
 '_id': ObjectId('61d0fc2e5a40174705eaf178'),
 'id_Parking': 6}
{'Average_speed': 74,
 'Card_id': 8565882626516635,
 'Departure_waiting_time': '2020-05-25 01:42:02',
 'Rented_vehicle_id': 1,
 'Status_id': True,
 'Vehicle_id': 5,
 '_id': ObjectId('61d0fc2e5a40174705eaf17b'),
 'id_Parking': 7}
{'Average_speed': 82,
 'Card_id': 7778788977096592,
 'Departure_waiting_time': '2020-05-25 02:19:39',
 'Rented_vehicle_id': 2,
 'Status_id': True,
 'Vehicle_id': 6,
 '_id': ObjectId('61d0fc2e5a40174705eaf17e'),
 'id_Parking': 6}
{'Average_speed': 7,
 'Card_id': 3900947831478853,
 'Departure_waiting_time': '2020-05-25 02:25:55',
 'Rented_vehicle_id': 18,
 'Status_id': True,
 'Vehicle_id': 22,
 '_id': ObjectId('61d0fc2e5a40174705eaf17f'),
 'id_Parking': 12}
{'Average_speed': 24,
 'Card_id': 2983575090750656,
 'Departure_waiting_time': '2

In [ ]:
### Query 2
### Вывести уникальные номера карт по которым средняя скорость арендованного транспортного средства
### больше либо равна 85 км/ч
result = db.vehicle_rent_status_data.aggregate(
    [{"$match": {"Average_speed": {"$gte": 85}}},
     {"$project": {"_id": 0, "Card_id": 1}},
     {"$group": {"_id": None, "Card_id": {"$addToSet": "$Card_id"}}},
     {"$unwind": "$Card_id" },
     {"$project": {"_id": 0 }},
    ]
)

for doc in result:
  pprint(doc)

{'Card_id': 5432211932197410}
{'Card_id': 9114176625492216}
{'Card_id': 1725898220879862}
{'Card_id': 7779737422968947}
{'Card_id': 8969212414668139}
{'Card_id': 4675954164635347}
{'Card_id': 8482208593266279}
{'Card_id': 4914853618400966}
{'Card_id': 6491526344317502}
{'Card_id': 4525254644942707}
{'Card_id': 5199929144419664}
{'Card_id': 4737889807427200}
{'Card_id': 4043667536269564}
{'Card_id': 9370794525715332}
{'Card_id': 9682258487383144}
{'Card_id': 7266881448465289}
{'Card_id': 3752081031680182}
{'Card_id': 9678177036976606}
{'Card_id': 1456021440640112}
{'Card_id': 4173454651770605}
{'Card_id': 4616000909000636}
{'Card_id': 2149400198971294}
{'Card_id': 6458926390352590}
{'Card_id': 3523013776655322}
{'Card_id': 1944014343380941}
{'Card_id': 8728682035327593}
{'Card_id': 3479625106304534}
{'Card_id': 8858153974505284}
{'Card_id': 7673424858295755}
{'Card_id': 4609024382763565}
{'Card_id': 1631065955041969}
{'Card_id': 9316393769640842}
{'Card_id': 2433299760737735}
{'Card_id'

In [ ]:
### Query 3
### Выбрать топ 3 парковок по вместимости для каждого типа транспорта.
f_part = db.parking.aggregate(
    [{"$group" : {"_id": "$Transport_type_id"}},
     {"$sort" : {"_id": ASCENDING}}
     ]
)
for doc in f_part:
  s_part = db.parking.aggregate(
      [{"$match": {"Transport_type_id": doc['_id']}},
       {"$project": {"_id": 0, "id_Parking": 1, "Parking_address": 1, "Capacity": 1, "Transport_type_id": 1}},
       {"$sort": {"Capacity": DESCENDING}},
       {"$limit": 3}
      ]
  )
  for sdoc in s_part:
    print(sdoc)

{'id_Parking': 19, 'Parking_address': 'Русаковская набережная, 1с2', 'Capacity': 16, 'Transport_type_id': 0}
{'id_Parking': 20, 'Parking_address': 'Сельскохозяйственная улица, 9с3', 'Capacity': 14, 'Transport_type_id': 0}
{'id_Parking': 21, 'Parking_address': 'Анненская ул., 25', 'Capacity': 825, 'Transport_type_id': 1}
{'id_Parking': 23, 'Parking_address': 'Москва, Ленинградское шоссе, д.4', 'Capacity': 810, 'Transport_type_id': 1}
{'id_Parking': 22, 'Parking_address': 'ул. Ходынская, вл. 5', 'Capacity': 680, 'Transport_type_id': 1}
{'id_Parking': 17, 'Parking_address': 'Варшавское шоссе, 95', 'Capacity': 15, 'Transport_type_id': 4}
{'id_Parking': 18, 'Parking_address': 'Рязанский проспект, 105', 'Capacity': 14, 'Transport_type_id': 4}
{'id_Parking': 16, 'Parking_address': 'Бесединское ш., 17, стр. 1', 'Capacity': 12, 'Transport_type_id': 4}
{'id_Parking': 2, 'Parking_address': 'ул. Щепкина, 49', 'Capacity': 1684, 'Transport_type_id': 5}
{'id_Parking': 3, 'Parking_address': 'ул. Гиляр

In [ ]:
### Query 4
### Выбрать топ 3 транспортных средства с наибольшим количеством данных валидатора для каждого типа транспорта.
f_part = db.transport.aggregate(
    [{"$group" : {"_id": "$Transport_type_id"}},
     {"$sort" : {"_id": ASCENDING}}
     ]
)
for transport_type_doc in f_part:
  test = db.transport.aggregate(
          [{"$lookup":{
              "from": "validator_s_data",
              "localField": "Vehicle_id",
              "foreignField": "Vehicle_id",
              "as" : "valisators_data"
          }},
          {"$match": {"Transport_type_id": transport_type_doc['_id']}},
          {"$project": {"_id": 0, "Vehicle_id": 1, "VCOUNT": { "$size":"$valisators_data" }, "Transport_type_id": 1}},
          {"$sort": {"VCOUNT": DESCENDING}},
          {"$limit": 3}
          ])
  for doc in test:
    pprint(doc)

{'Transport_type_id': 1, 'VCOUNT': 8015, 'Vehicle_id': 4}
{'Transport_type_id': 1, 'VCOUNT': 7923, 'Vehicle_id': 2}
{'Transport_type_id': 1, 'VCOUNT': 7800, 'Vehicle_id': 3}
{'Transport_type_id': 5, 'VCOUNT': 0, 'Vehicle_id': 5}
{'Transport_type_id': 5, 'VCOUNT': 0, 'Vehicle_id': 6}
{'Transport_type_id': 5, 'VCOUNT': 0, 'Vehicle_id': 7}
{'Transport_type_id': 6, 'VCOUNT': 0, 'Vehicle_id': 16}
{'Transport_type_id': 6, 'VCOUNT': 0, 'Vehicle_id': 17}
{'Transport_type_id': 6, 'VCOUNT': 0, 'Vehicle_id': 18}


In [ ]:
### Приведение координат к нужному формату для 5 запроса
for doc in db.sensors_data.find():
  db.sensors_data.update_one({'_id': {"$eq": doc['_id']}}, 
                             {"$set" : {"location": [ doc['Geolocation_coordinates_X'], doc['Geolocation_coordinates_Y']]}})

db.sensors_data.create_index([("location", GEOSPHERE)])

'location_2dsphere'

In [ ]:
for doc in db.sensors_data.find().limit(1):
  pprint(doc)

{'Data_accuracy': 1,
 'Door_closing_time': '2020-05-25 08:00:12',
 'Door_opening_time': '2020-05-25 08:00:00',
 'Geolocation_coordinates_X': 55.820667,
 'Geolocation_coordinates_Y': 37.663939,
 'Passengers_came_in': 8,
 'Passengers_got_off': 0,
 'Public_vehicle_id': 1,
 'Record_id': 1,
 'Vehicle_id': 1,
 '_id': ObjectId('61d0f9225a40174705e8cbdc'),
 'location': [55.820667, 37.663939]}


In [ ]:
# Проверка самописной функцией
def geo_dist(X_start,Y_start, X_end, Y_end):
  dist = 6371302 * 2 * math.asin(math.sqrt(math.pow(math.sin((X_start - abs(X_end)) * math.pi/180 / 2), 2) + 
                                           math.cos(X_start * math.pi/180) * 
                                           math.cos(abs(X_end) * math.pi/180) * 
                                           math.pow(math.sin((Y_start - Y_end) * math.pi/ 180/ 2), 2)))
  return dist


for stop in db.a_stop.find():
  counter = 0
  for sensor_note in db.sensors_data.find():
    
    if geo_dist(stop["Geolocation_coordinates_X"], stop['Geolocation_coordinates_Y'], sensor_note["Geolocation_coordinates_X"], sensor_note['Geolocation_coordinates_Y']) <= 5:
      counter += sensor_note['Passengers_came_in']

  print(stop['Stop_id'],'  =  ', counter)

11   =   3267
12   =   3375
13   =   1625
14   =   1696
15   =   1574
16   =   1653
17   =   1643
18   =   1708
19   =   1614
20   =   1754
21   =   0
22   =   1753
23   =   1635
24   =   1608
25   =   1749
26   =   1656
27   =   1751
28   =   1647
29   =   1677


In [ ]:
### Query 5
### Получить топ 5 остановок, где садится наибольшее количество людей.
### Для этого запроса понадобиться дополнительная встроенная функция geoNear() для расчета расстояния между двумя точками.
subresult = []
for doc in db.a_stop.find():
  cc = db.sensors_data.aggregate(
    [{"$geoNear":{"near": {"type": "Point", "coordinates": [doc['Geolocation_coordinates_X'], doc['Geolocation_coordinates_Y']]},
                  "distanceField": "dist",
                  "spherical": True,
                  "key": "location",
                  "maxDistance": 5
                }
      },
     {"$group": {"_id": None, "Passengers": {"$sum": "$Passengers_came_in"}}}
    ])
  for dd in cc:
    subresult.append({"Stop_id": doc['Stop_id'], "Name": doc['Name'], "Passengers": int(dd['Passengers'])})

j_s = json.dumps(subresult, ensure_ascii=False)
j_s = json.loads(j_s)

db.third_query_sub.insert_many(j_s)

result = db.third_query_sub.aggregate(
    [{"$project": {"_id": 0, "Name": 1, "Passengers": 1, "Stop_id": 1}},
     {"$sort": {"Passengers": DESCENDING}},
     {"$limit": 5}
    ]
)
for doc in result:
  pprint(doc)

{'Name': 'Улица Павла Корчагина, 5', 'Passengers': 3375, 'Stop_id': 12}
{'Name': 'Улица Павла Корчагина', 'Passengers': 3267, 'Stop_id': 11}
{'Name': 'Улица Павла Корчагина, 8 – Платформа Маленковская',
 'Passengers': 1754,
 'Stop_id': 20}
{'Name': 'Улица Кибальчича', 'Passengers': 1753, 'Stop_id': 22}
{'Name': 'Институт биологии и химии', 'Passengers': 1751, 'Stop_id': 27}


In [ ]:
for i in db.third_query_sub.find({}):
  pprint(i)

{'Name': 'Улица Павла Корчагина',
 'Passengers': 3267,
 'Stop_id': 11,
 '_id': ObjectId('61d0f9295a40174705e95688')}
{'Name': 'Улица Павла Корчагина, 5',
 'Passengers': 3375,
 'Stop_id': 12,
 '_id': ObjectId('61d0f9295a40174705e95689')}
{'Name': '1-й Рижский переулок',
 'Passengers': 1625,
 'Stop_id': 13,
 '_id': ObjectId('61d0f9295a40174705e9568a')}
{'Name': 'Метро Алексеевская',
 'Passengers': 1696,
 'Stop_id': 14,
 '_id': ObjectId('61d0f9295a40174705e9568b')}
{'Name': 'Крестовский мост',
 'Passengers': 1574,
 'Stop_id': 15,
 '_id': ObjectId('61d0f9295a40174705e9568c')}
{'Name': 'Рижский вокзал',
 'Passengers': 1653,
 'Stop_id': 16,
 '_id': ObjectId('61d0f9295a40174705e9568d')}
{'Name': 'Графский переулок',
 'Passengers': 1643,
 'Stop_id': 17,
 '_id': ObjectId('61d0f9295a40174705e9568e')}
{'Name': 'Метро Алексеевская',
 'Passengers': 1708,
 'Stop_id': 18,
 '_id': ObjectId('61d0f9295a40174705e9568f')}
{'Name': '1-й Рижский переулок',
 'Passengers': 1614,
 'Stop_id': 19,
 '_id': Object